In [1]:
import rasterio
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import datetime
import glob as glob
import datetime
import scipy
import sys
import os
import geopandas as gp
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
# import seaborn as sns
%matplotlib inline

## Rainfall dataset

We use the rainfall dataset produced by Levy et al. (2017):  

Levy, M. C. (2017). Curated rain and flow data for the Brazilian rainforest-savanna transition zone, HydroShare, http://www.hydroshare.org/resource/e82e66572b444fc5b6bf16f88f911f77

The data is interpolated using each day's active rain gauge data. Interpolations were done on a 0.25° resolution grid; the IDW parameter was set to 2 (default). The maximum interpolation distance (radius) was 1000m. The dataset was then sampled at all 5000 datapoints in `./data/sesync2018_points.geojson`, which were randomly generated (avoiding 'artificial surfaces' using GlobCover 2009, a global land cover map available in Google Earth Engine) within the Amazon and Cerrado biomes (note: some points were discarded because the rainfall dataset does not entirely cover the extent of the Amazon and Cerrado biomes). 

In [138]:
df = pd.read_csv('./data/sesync2018_points_rainIdw.csv', parse_dates=True, index_col=0)

In [139]:
rainfall_threshold = 2.0
# get frequency of rainfall occurence
GBL=df.groupby([(df.index.year),(df.index.month)]).apply(
      lambda x: np.sum(x>rainfall_threshold)/float(len(x))
)

    
GBA=df.groupby([(df.index.year),(df.index.month)]).apply(
    lambda x: np.mean(x[x>rainfall_threshold]) 
)

# get lambdas pre/post treatment
pre = [1980, 1990]
post = [2004, 2014]



In [298]:
# read forest loss data
loss = pd.read_csv('data/hansen/loss.csv')
loss['id'] = loss['id'].astype(str)


GBLpre = GBL.loc[pre[0]:pre[1]].groupby(level=[1]).mean().T
GBLpre['id'] = GBLpre.index
GBLpre['post'] = 0
GBLpre = GBLpre.merge(loss, left_on='id', right_on='id', how='left'
                      ).melt(id_vars=['id','post', 'treated'], value_name='L', var_name='month')


GBLpost = GBL.loc[post[0]:post[1]].groupby(level=[1]).mean().T
GBLpost['id'] = GBLpost.index
GBLpost['post'] = 1
GBLpost = GBLpost.merge(loss, left_on='id', right_on='id', how='left'
                       ).melt(id_vars=['id','post', 'treated'], value_name='L', var_name='month')
tofit = pd.concat([GBLpost, GBLpre])
x1 = tofit.post.values
x2 = tofit.treated.values
tofit['treatmentstatus'] = 0
tofit['treatmentstatus'] = x2*x1
tofit = tofit.set_index(['id', 'post'])


for g in tofit.groupby('month'):
    mi_data = pd.DataFrame(g[1])
    exog_vars = ['treatmentstatus']
    exog = sm.add_constant(mi_data[exog_vars])
    mod = PanelOLS(mi_data.L, exog, entity_effects=True, time_effects=True)
    print('For month %s \n'%(str(g[0])))
    print(mod.fit())


For month 1 

                          PanelOLS Estimation Summary                           
Dep. Variable:                      L   R-squared:                        0.0034
Estimator:                   PanelOLS   R-squared (Between):              0.0015
No. Observations:                7824   R-squared (Within):               0.0008
Date:                Mon, Apr 02 2018   R-squared (Overall):              0.0014
Time:                        17:14:27   Log-likelihood                 1.384e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      13.218
Entities:                        3912   P-value                           0.0003
Avg Obs:                       2.0000   Distribution:                  F(1,3910)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             13.218
              

                          PanelOLS Estimation Summary                           
Dep. Variable:                      L   R-squared:                        0.0029
Estimator:                   PanelOLS   R-squared (Between):              0.0017
No. Observations:                7824   R-squared (Within):              -0.0048
Date:                Mon, Apr 02 2018   R-squared (Overall):              0.0015
Time:                        17:14:29   Log-likelihood                 1.187e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      11.368
Entities:                        3912   P-value                           0.0008
Avg Obs:                       2.0000   Distribution:                  F(1,3910)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             11.368
                            

                          PanelOLS Estimation Summary                           
Dep. Variable:                      L   R-squared:                        0.0139
Estimator:                   PanelOLS   R-squared (Between):              0.0086
No. Observations:                7824   R-squared (Within):               0.0461
Date:                Mon, Apr 02 2018   R-squared (Overall):              0.0110
Time:                        17:14:31   Log-likelihood                  1.44e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      55.086
Entities:                        3912   P-value                           0.0000
Avg Obs:                       2.0000   Distribution:                  F(1,3910)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             55.086
                            

In [297]:
# read forest loss data
loss = pd.read_csv('data/hansen/loss.csv')
loss['id'] = loss['id'].astype(str)


GBApre = GBA.loc[pre[0]:pre[1]].groupby(level=[1]).mean().T
GBApre['id'] = GBApre.index
GBApre['post'] = 0
GBApre = GBApre.merge(loss, left_on='id', right_on='id', how='left'
                      ).melt(id_vars=['id','post', 'treated'], value_name='A', var_name='month')


GBApost = GBA.loc[post[0]:post[1]].groupby(level=[1]).mean().T
GBApost['id'] = GBApost.index
GBApost['post'] = 1
GBApost = GBApost.merge(loss, left_on='id', right_on='id', how='left'
                       ).melt(id_vars=['id','post', 'treated'], value_name='A', var_name='month')
tofit = pd.concat([GBApost, GBApre])
x1 = tofit.post.values
x2 = tofit.treated.values
tofit['treatmentstatus'] = 0
tofit['treatmentstatus'] = x2*x1
tofit = tofit.set_index(['id', 'post'])



for g in tofit.groupby('month'):
    mi_data = pd.DataFrame(g[1])
    exog_vars = ['treatmentstatus']
    exog = sm.add_constant(mi_data[exog_vars])
    mod = PanelOLS(mi_data.A, exog, entity_effects=True, time_effects=True)
    print('\n\n+++++++++++++++++++++\n+++++++++++++++++++++\nFor month %s \n'%(str(g[0])))
    print(mod.fit())







+++++++++++++++++++++
+++++++++++++++++++++
For month 1 

                          PanelOLS Estimation Summary                           
Dep. Variable:                      A   R-squared:                     7.736e-05
Estimator:                   PanelOLS   R-squared (Between):             -0.0007
No. Observations:                7824   R-squared (Within):              -0.0002
Date:                Mon, Apr 02 2018   R-squared (Overall):             -0.0006
Time:                        17:13:58   Log-likelihood                   -9202.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      0.3025
Entities:                        3912   P-value                           0.5823
Avg Obs:                       2.0000   Distribution:                  F(1,3910)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statist

                          PanelOLS Estimation Summary                           
Dep. Variable:                      A   R-squared:                        0.0033
Estimator:                   PanelOLS   R-squared (Between):             -0.0022
No. Observations:                7824   R-squared (Within):               0.0099
Date:                Mon, Apr 02 2018   R-squared (Overall):             -0.0006
Time:                        17:14:01   Log-likelihood                -1.067e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      12.832
Entities:                        3912   P-value                           0.0003
Avg Obs:                       2.0000   Distribution:                  F(1,3910)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             12.832
                            

/Users/daviddralle/anaconda2/envs/gr3/lib/python3.5/site-packages/linearmodels/utility.py:490: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)




+++++++++++++++++++++
+++++++++++++++++++++
For month 7 

                          PanelOLS Estimation Summary                           
Dep. Variable:                      A   R-squared:                        0.0033
Estimator:                   PanelOLS   R-squared (Between):             -0.0030
No. Observations:                7555   R-squared (Within):              -0.0011
Date:                Mon, Apr 02 2018   R-squared (Overall):             -0.0027
Time:                        17:14:02   Log-likelihood                -1.293e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      11.945
Entities:                        3911   P-value                           0.0006
Avg Obs:                       1.9317   Distribution:                  F(1,3642)
Min Obs:                       1.0000                                           
Max Obs:                       2.0000   F-statist

                          PanelOLS Estimation Summary                           
Dep. Variable:                      A   R-squared:                        0.0107
Estimator:                   PanelOLS   R-squared (Between):              0.0001
No. Observations:                7824   R-squared (Within):               0.0077
Date:                Mon, Apr 02 2018   R-squared (Overall):              0.0015
Time:                        17:14:05   Log-likelihood                -1.074e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      42.422
Entities:                        3912   P-value                           0.0000
Avg Obs:                       2.0000   Distribution:                  F(1,3910)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             42.422
                            

In [ ]:
from linearmodels import PanelOLS
test = pd.DataFrame.from_dict({0:Ss[0,:], 1:Ss[1,:]})
test['treated'] = np.random.randint(0,2,len(test))
test['id'] = df.columns
tofit = test.melt(value_vars = [0, 1], id_vars = ['treated', 'id'])
tofit['treatmentstatus'] = tofit.treated*tofit.variable
tofit = tofit.set_index(['id', 'variable'])
res = PanelOLS(tofit.value, tofit.treatmentstatus, entity_effects=True, time_effects=True).fit()
print(res)